In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
import tensorflow as tf
from policy import ConvNet
from concepts import DynamicConcepts, linear_regression
import absl.logging
import json
import string
import random
import numpy as np
import os
from env import gogame
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

session_name = "falcon"
model_name = "net"
board_size = 5
board_name = f'{board_size}x{board_size}'
agents_to_sample = [10, 20, 60, 100, 500, 1000]

# Mcts simulations
simulations = 1000

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

mcts_model_path = f"../models/best_models/board_size_{board_size}/net_1000.keras"

init_state, concept_type_single = DynamicConcepts.opening_play(board_size)

dynamic_concept = DynamicConcepts(
    init_state, simulations, board_size, concept_type_single, mcts_model_path)

CONCEPT_NAME = DynamicConcepts.opening_play.__name__

2024-02-09 13:55:08.953919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-09 13:55:12.200627: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-09 13:55:12.200675: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpdzcr9zly/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpdzcr9zly/assets
2024-02-09 13:55:16.515320: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpdzcr9zly
2024-02-09 13:55:16.517534: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-09 13:55:16.517550: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpdzcr9zly
2024-02-09 13:55:16.526700: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-09 13:55:16.570423: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpdzcr9zly
2024-02-09 13:55:16.586100: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 70783 microseconds.
INFO: Crea

In [3]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [4]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (9, 5, 5, 5)
Negative cases:  (21, 5, 5, 5)


In [5]:
# If larger than 50, do not visualize the tree do to the size (crashes the kernel)
if simulations <= 100:
    graph = dynamic_concept.view_tree()

    graph.render('./visualization/images/tree', view=True)

In [6]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [7]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [6]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print(gogame.str(negative_cases[i], nn_format=True))
    print()

Negative case 0:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]]
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─○─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝


Negative case 1:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]


In [13]:
BINARY = True

# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=True,
    epochs=50,
    dynamic=True
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/dynamic", exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Positions to consider:  44
Epoch 1/50


2024-02-09 13:38:17.467025: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:38:17.500839: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 1s 194ms/step - loss: 0.8935
Epoch 2/50
2/2 [==============================] - 0s 36ms/step - loss: 0.8926
Epoch 3/50
2/2 [==============================] - 0s 23ms/step - loss: 0.8917
Epoch 4/50
2/2 [==============================] - 0s 31ms/step - loss: 0.8908
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.8900
Epoch 6/50
2/2 [==============================] - 0s 33ms/step - loss: 0.8891
Epoch 7/50
2/2 [==============================] - 0s 22ms/step - loss: 0.8883
Epoch 8/50
2/2 [==============================] - 0s 23ms/step - loss: 0.8875
Epoch 9/50
2/2 [==============================] - 0s 33ms/step - loss: 0.8867
Epoch 10/50
2/2 [==============================] - 0s 23ms/step - loss: 0.8858
Epoch 11/50
2/2 [==============================] - 0s 22ms/step - loss: 0.8850
Epoch 12/50
2/2 [==============================] - 0s 22ms/step - loss: 0.8841
Epoch 13/50
2/2 [==============================] - 0s 22ms/step - loss:

2024-02-09 13:38:20.540870: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [14]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=50,
            dynamic=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/dynamic", exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)

Getting activation outputs: 100%|██████████| 2/2 [00:00<00:00, 26.96it/s]

Performing regression for layer 0
Epoch 1/50



2024-02-09 13:38:26.651160: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:38:26.691255: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 1s 168ms/step - loss: 1.1041
Epoch 2/50
2/2 [==============================] - 0s 32ms/step - loss: 1.0998
Epoch 3/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0962
Epoch 4/50
2/2 [==============================] - 0s 30ms/step - loss: 1.0918
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0883
Epoch 6/50
2/2 [==============================] - 0s 35ms/step - loss: 1.0842
Epoch 7/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0805
Epoch 8/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0769
Epoch 9/50
2/2 [==============================] - 0s 22ms/step - loss: 1.0730
Epoch 10/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0695
Epoch 11/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0658
Epoch 12/50
2/2 [==============================] - 0s 46ms/step - loss: 1.0624
Epoch 13/50
2/2 [==============================] - 0s 22ms/step - loss:

2024-02-09 13:38:29.661736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:38:30.042572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:38:30.077310: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

2024-02-09 13:39:23.170748: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:39:23.534717: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:39:23.567324: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 1s 193ms/step - loss: 1.1484
Epoch 2/50
2/2 [==============================] - 0s 40ms/step - loss: 1.1411
Epoch 3/50
2/2 [==============================] - 0s 24ms/step - loss: 1.1345
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 1.1284
Epoch 5/50
2/2 [==============================] - 0s 33ms/step - loss: 1.1219
Epoch 6/50
2/2 [==============================] - 0s 30ms/step - loss: 1.1159
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 1.1096
Epoch 8/50
2/2 [==============================] - 0s 33ms/step - loss: 1.1039
Epoch 9/50
2/2 [==============================] - 0s 21ms/step - loss: 1.0977
Epoch 10/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0924
Epoch 11/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0862
Epoch 12/50
2/2 [==============================] - 0s 22ms/step - loss: 1.0808
Epoch 13/50
2/2 [==============================] - 0s 22ms/step - loss:

2024-02-09 13:39:26.496803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:39:26.855764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 13:39:26.888040: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 1s 183ms/step - loss: 1.1542
Epoch 2/50
2/2 [==============================] - 0s 23ms/step - loss: 1.1497
Epoch 3/50
2/2 [==============================] - 0s 38ms/step - loss: 1.1454
Epoch 4/50
2/2 [==============================] - 0s 38ms/step - loss: 1.1412
Epoch 5/50
2/2 [==============================] - 0s 30ms/step - loss: 1.1371
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 1.1330
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 1.1290
Epoch 8/50
2/2 [==============================] - 0s 23ms/step - loss: 1.1248
Epoch 9/50
2/2 [==============================] - 0s 22ms/step - loss: 1.1208
Epoch 10/50
2/2 [==============================] - 0s 23ms/step - loss: 1.1167
Epoch 11/50
2/2 [==============================] - 0s 30ms/step - loss: 1.1128
Epoch 12/50
2/2 [==============================] - 0s 30ms/step - loss: 1.1087
Epoch 13/50
2/2 [==============================] - 0s 22ms/step - loss:

2024-02-09 13:39:29.778349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
